In [ ]:
import numpy as np
import pandas as pd
import selenium.webdriver
from bs4 import BeautifulSoup

In [ ]:
browser = selenium.webdriver.Edge()
browser.get("https://www.bing.com/homes/map?formcode=000074&q=mcallen%20and%20edinburg&tab=sale")

In [ ]:
soup = BeautifulSoup(browser.page_source)

In [ ]:
houses = soup.select("a[class^='HouseList_house_item']")
data = pd.DataFrame({
    "Price": [i.select_one("span[class^='HouseList_house_price']").text for i in houses],
    "RoomInfo": [i.select_one("div[class^='HouseList_house_roominfo']").text for i in houses],
    "Address": [i.select_one("img[class^='HouseList_house_img']").attrs['alt'] for i in houses],
    "href": ["https://www.bing.com" + i.attrs['href'] for i in houses],
})
data["Price"] = data["Price"].str.replace("[^0-9]","",regex=True).astype('int')
data["#Beds"] = data["RoomInfo"].str.findall("^.*bed").str.join("").str.replace("[^0-9]","",regex=True).replace("^\s*$", np.nan, regex=True).astype('float')
data["#Baths"] = data["RoomInfo"].str.findall(",.*bath").str.join("").str.replace("[^0-9.]","",regex=True).replace("^\s*$", np.nan, regex=True).astype('float')
data["Area"] = data["RoomInfo"].str.findall("bath, .*sqft").str.join("").str.replace("[^0-9]","",regex=True).replace("^\s*$", np.nan, regex=True).astype('float')
data["Zip"] = data["Address"].str.findall(", TX .*").str.join("").str.replace("[^0-9]","",regex=True).replace("^\s*$", np.nan, regex=True).astype('int')
data["PricePerFoot"] = data["Price"] / data["Area"] 
data = data.dropna()
data = data[data["#Beds"] > 0]
data = data[data["#Baths"] > 0]
data.to_pickle("data.pkl")
data